# 2. Building the DeepAR Model: Data Preprocessing, Hyperparameter Search, and Model Training

In this script, we go through the steps of building a DeepAR forecasting model. We  introduce several classes which will guide us along proces, with the aim of unlocking the potential of DeepAR for accurate and insightful forecasting.

## 1. Load the Data

In [1]:
import mxnet as mx
from helper_functions import Helper
from agents import Evaluation_Agent
from agents import Preparation_Agent
from agents import Activity_Agent, Usage_Agent, Load_Agent, Price_Agent

helper = Helper()


Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [2]:
devices = {0: {'hh': 'hh1', 'dev_name': 'Washing Machine', 'dev': 'washing_machine'},
          1: {'hh': 'hh1', 'dev_name': 'Dishwasher', 'dev': 'dishwasher'},
          2: {'hh': 'hh2', 'dev_name': 'Washing Machine', 'dev': 'washing_machine'},
          3: {'hh': 'hh2', 'dev_name': 'Dishwasher', 'dev': 'dishwasher'},
          4: {'hh': 'hh3', 'dev_name': 'Tumble Dryer', 'dev': 'tumble_dryer'},
          5: {'hh': 'hh3', 'dev_name': 'Washing Machine', 'dev': 'washing_machine'},
          6: {'hh': 'hh3', 'dev_name': 'Dishwasher', 'dev': 'dishwasher'},
          7: {'hh': 'hh4', 'dev_name': 'Washing Machine (1)', 'dev': 'washing_machine'},
          8: {'hh': 'hh4', 'dev_name': 'Washing Machine (2)', 'dev': 'washing_machine'},
          9: {'hh': 'hh5', 'dev_name': 'Tumble Dryer', 'dev': 'tumble_dryer'},
          10: {'hh': 'hh6', 'dev_name': 'Washing Machine', 'dev': 'washing_machine'},
          11: {'hh': 'hh6', 'dev_name': 'Dishwasher', 'dev': 'dishwasher'},
          12: {'hh': 'hh7', 'dev_name': 'Tumble Dryer', 'dev': 'tumble_dryer'},
          13: {'hh': 'hh7', 'dev_name': 'Washing Machine', 'dev': 'washing_machine'},
          14: {'hh': 'hh7', 'dev_name': 'Dishwasher', 'dev': 'dishwasher'},
          15: {'hh': 'hh8', 'dev_name': 'Washing Machine', 'dev': 'washing_machine'},
          16: {'hh': 'hh9', 'dev_name': 'Washer Dryer', 'dev': 'washer_dryer'},
          17: {'hh': 'hh9', 'dev_name': 'Washing Machine', 'dev': 'washing_machine'},
          18: {'hh': 'hh9', 'dev_name': 'Dishwasher', 'dev': 'dishwasher'},
          19: {'hh': 'hh10', 'dev_name': 'Washing Machine', 'dev': 'washing_machine'},
          }



In [3]:
from helper_functions_thesis import Helper_Functions_Thesis
shiftable_devices_dict = Helper_Functions_Thesis.create_shiftable_devices_dict(devices)

shiftable_devices_dict


{'hh1': ['Washing Machine', 'Dishwasher'],
 'hh2': ['Washing Machine', 'Dishwasher'],
 'hh3': ['Tumble Dryer', 'Washing Machine', 'Dishwasher'],
 'hh4': ['Washing Machine (1)', 'Washing Machine (2)'],
 'hh5': ['Tumble Dryer'],
 'hh6': ['Washing Machine', 'Dishwasher'],
 'hh7': ['Tumble Dryer', 'Washing Machine', 'Dishwasher'],
 'hh8': ['Washing Machine'],
 'hh9': ['Washer Dryer', 'Washing Machine', 'Dishwasher'],
 'hh10': ['Washing Machine']}

In [4]:
active_appliences_dict_complete = {
    'hh1' : ['Washing Machine', 'Dishwasher', 'Computer Site', 'Television Site', 'Electric Heater'], #'Tumble Dryer', 
    'hh2' : ['Washing Machine', 'Dishwasher', 'Television', 'Microwave', 'Toaster', 'Hi-Fi', 'Kettle', 'Oven Extractor Fan'],
    'hh3' : ['Tumble Dryer', 'Washing Machine', 'Dishwasher','Toaster', 'Television', 'Microwave', 'Kettle'],
    'hh4' : ['Washing Machine (1)', 'Washing Machine (2)', 'Computer Site', 'Television Site', 'Microwave', 'Kettle'],
    'hh5' : ['Tumble Dryer', 'Computer Site', 'Television Site', 'Combination Microwave', 'Kettle', 'Toaster'], # , 'Washing Machine' --> consumes energy constantly; , 'Dishwasher' --> noise at 3am
    'hh6' : ['Washing Machine', 'Dishwasher', 'MJY Computer', 'Television Site', 'Microwave', 'Kettle', 'Toaster', 'PGM Computer'],
    'hh7' : ['Tumble Dryer', 'Washing Machine', 'Dishwasher', 'Television Site', 'Toaster', 'Kettle'],
    'hh8' : ['Washing Machine', 'Toaster', 'Computer', 'Television Site', 'Microwave', 'Kettle'], # 'Dryer' --> consumes constantly
    'hh9' : ['Washer Dryer', 'Washing Machine', 'Dishwasher', 'Television Site', 'Microwave', 'Kettle', 'Hi-Fi', 'Electric Heater'], 
    'hh10' : ['Washing Machine', 'Magimix (Blender)','Television Site', 'Microwave', ' Kenwood KMix'] #'Dishwasher'
}

active_appliences_dict = {key: active_appliences_dict_complete[key] for key in shiftable_devices_dict}
active_appliences_dict.keys()


dict_keys(['hh1', 'hh2', 'hh3', 'hh4', 'hh5', 'hh6', 'hh7', 'hh8', 'hh9', 'hh10'])

In [5]:
import os
DATA_PATH = os.getcwd()+'/data/'
PICKLE_PATH = os.getcwd()+'/data/pickle_files'


In [6]:
threshold = 0.01
shiftable_devices = ['Tumble Dryer', 'Washing Machine', 'Dishwasher'] 
active_appliances = ['Toaster', 'Tumble Dryer', 'Dishwasher', 'Washing Machine','Television', 'Microwave', 'Kettle']

#load_parameters
truncation_params = {
    'features': 'all', 
    'factor': 1.5, 
    'verbose': 1
}

scale_params = {
    'features': 'all', 
    'kind': 'MinMax', 
    'verbose': 1
}

aggregate_params = {
    'resample_param': '60T'
}

device_params = {
    'threshold': 0.15
}

load_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'aggregate': aggregate_params,
    'shiftable_devices': shiftable_devices, 
    'device': device_params
}

#activity_parameters

activity_params = {
    'active_appliances': active_appliances,
    'threshold': threshold 
}

time_params = {
    'features': ['hour', 'day_name']
}

activity_lag_params = {
    'features': ['activity'],
    'lags': [24, 48, 72]
}

activity_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'aggregate': aggregate_params,
    'activity': activity_params,
    'time': time_params,
    'activity_lag': activity_lag_params
}

#usage_parameters

device = {
    'threshold' : threshold}

aggregate_params24_H = {
    'resample_param': '24H'
}

usage_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'activity': activity_params,
    'aggregate_hour': aggregate_params,
    'aggregate_day': aggregate_params24_H,
    'time': time_params,
    'activity_lag': activity_lag_params,
    'shiftable_devices' : shiftable_devices,
    'device': device
}


We begin the script by generating outputs from the Usage Agent, Availability Agent, and Load Agent, which contain variables that we will use to train the forecasting model. To expedite the process, pre-generated outputs for all households can be uploaded from the cell below.

In [7]:
load_dict = Helper_Functions_Thesis.open_pickle_file(PICKLE_PATH, 'load_dict.pickle')
activity_dict = Helper_Functions_Thesis.open_pickle_file(PICKLE_PATH, 'activity_dict.pickle')
usage_dict = Helper_Functions_Thesis.open_pickle_file(PICKLE_PATH, 'usage_dict.pickle')


## 2. Data Preparation

We build the class **Create_Dataset** to construct datasets for each device with all the necessary information to train the forecasting model. We use three agents from the smart home system (load, usage and activity), to gather essential features.

### 2.1 Initialize the Class

The first step is to initialize the **Create_Dataset** class with the necessary parameters.

In [8]:
start_dataset = '2014-03-01'
end_dataset = '2015-05-07'


In [9]:
from typing import Dict, List, Any, Tuple
import pandas as pd
import numpy as np

class Create_Dataset:
    '''
    A class for constructing datasets for each device with all the necessary information to train
    the forecasting model.
    Parameters:
    ----------
    start_dataset:
        A string ('yyyy-mm-dd') to determine the start of the dataset.
    end_dataset:
        A string ('yyyy-mm-dd') to determine the end of the dataset.
    devices:
        A dictionary containing device-specific information.
    load_dict:
        A dictionary containing the output dataframes from the Load Agent for each household.
    usage_dict:
        A dictionary containing the output dataframes from the Usage Agent for each household.
    activity_dict:
        A dictionary containing the output dataframes from the Activity Agent for each household.
    fill_na:
        Bool, controls if missing values should be filled with zeros.
    '''

    def __init__(
        self,
        start_dataset: str,
        end_dataset: str,
        devices: Dict[str, Any],
        load_dict: Dict[str, pd.DataFrame],
        usage_dict: Dict[str, pd.DataFrame],
        activity_dict: Dict[str, pd.DataFrame],
        fill_na = True,
    ):
        from helper_functions_thesis import Helper_Functions_Thesis
        self.start_dataset = start_dataset
        self.end_dataset = end_dataset
        self.devices = devices
        self.load_dict = load_dict
        self.usage_dict = usage_dict
        self.activity_dict = activity_dict
        self.hh_list: List[str] = list(Helper_Functions_Thesis.create_shiftable_devices_dict(self.devices).keys())
        self.fill_na = fill_na


In [10]:
create_dataset = Create_Dataset(start_dataset, end_dataset, devices, load_dict, usage_dict, activity_dict)

### 2.2. Slice the Data

The **slice_datasets** method truncates the load, usage, and activity datasets for each household to the specified time interval defined by start_dataset and end_dataset. 

In [11]:
def slice_datasets(self) -> None:
    import pandas as pd

    for hh in self.hh_list:
        self.load_dict[hh] = self.load_dict[hh].loc[self.start_dataset:self.end_dataset, :]
        self.usage_dict[hh] = self.usage_dict[hh].loc[self.start_dataset:self.end_dataset, :]
        self.activity_dict[hh] = self.activity_dict[hh].loc[self.start_dataset:self.end_dataset, :]
        
setattr(Create_Dataset, 'slice_datasets', slice_datasets)
del slice_datasets

In [12]:
create_dataset.slice_datasets()

### 2.3 Fill NA Values

The **'fill_na_function'** method replaces NA values of the Agent outputs with zeros. The method is run if the *'fill_na'* variable is set to **True**

In [13]:
def fill_na_function(self) -> None:
    import pandas as pd

    for hh in self.hh_list:
        self.load_dict[hh] = self.load_dict[hh].fillna(0)
        self.usage_dict[hh] = self.usage_dict[hh].fillna(0)
        self.activity_dict[hh] = self.activity_dict[hh].fillna(0)
        
setattr(Create_Dataset, 'fill_na_function', fill_na_function)
del fill_na_function


In [14]:
if create_dataset.fill_na:
    create_dataset.fill_na_function()
    

### 2.4 Feature Engineering 

We will train the forecasting model with the following variables:
1. periods_since_last_activity
2. periods_since_last_usage
3. activity_prob
4. usage_prob
5. temp (temparature)
6. dwpt (dew point)
7. rhum (humidity)
8. wdir (wind direction)
9. wspd (wind speed)
10. hh (household id, static variable)
11. dev (type of device, static variable)

As one might notice, there are no time features explicitly present in the list of variables, despite them having considerable influence on any forecasting task. This is attributed to the capabilitiy of the DeepAR to autonomously generate and integrate time-related features, capturing temporal dependencies without manual intervention. This property of DeepAR allows the model to learn various temporal patterns and enhances prediction power while simplifying the dataset preparation process.

#### 2.4.1 Compute activity and usage probabilities

First step is to create the usage and activity probability variables using the corresponding agents from the recommendation system. These variables offer a more nuanced perspective beyond historical usage patterns.

In [15]:
def create_activity_probability(self, activity_dict: pd.DataFrame) -> np.ndarray:
    import numpy as np
    from datetime import timedelta
    from agents import Activity_Agent

    activity = Activity_Agent(activity_dict)
    X_train, y_train, X_test, y_test = activity.train_test_split(
        activity_dict, str(activity_dict.index[-1].date() + timedelta(1))
    )
    X_train = X_train.fillna(0)
    model = activity.fit(X_train, y_train, 'random forest')
    activity_probability = model.predict_proba(X_train)[:, 1]
    return activity_probability

setattr(Create_Dataset, 'create_activity_probability', create_activity_probability)
del create_activity_probability


In [16]:
def create_usage_probability(self, usage_dict: pd.DataFrame, dev_name: str) -> np.ndarray:
    import numpy as np
    from agents import Usage_Agent

    usage = Usage_Agent(usage_dict, dev_name)
    X_train, y_train, X_test, y_test = usage.train_test_split(
        usage_dict, self.end_dataset, train_start=self.start_dataset
    )
    X_train = X_train.fillna(0) 
    model = usage.fit(X_train, y_train.values.ravel(), 'random forest')
    usage_probability = model.predict_proba(X_train)[:, 1]
    return usage_probability

setattr(Create_Dataset, 'create_usage_probability', create_usage_probability)
del create_usage_probability


#### 2.4.2 Convert Daily Data to Hourly Granularity

There are three variables (periods_since_last_activity, periods_since_last_usage, usage_prob) which capture daily trends in device usage dynamics. To align these variables with the hourly granularity required for forecasting, we employ the **resample_daily_to_hourly_data** function. This method transforms the daily dataframes into hourly resolution.

In [17]:
def resample_daily_to_hourly_data(self, daily_data: pd.DataFrame) -> pd.DataFrame:
    import pandas as pd
    from agents import Activity_Agent

    dummy_day = daily_data.index[-1] + pd.Timedelta(days=1)
    dummy_row = pd.DataFrame(index=pd.DatetimeIndex([dummy_day]))
    daily_data_with_dummy = pd.concat([daily_data, dummy_row])
    hourly_data = daily_data_with_dummy.resample("H").ffill().iloc[:-1, :]
    return hourly_data

setattr(Create_Dataset, 'resample_daily_to_hourly_data', resample_daily_to_hourly_data)
del resample_daily_to_hourly_data


#### 2.4.3 Create Feature Datasets for Each Device

Now that we have all the necessary functions at hand to generate the necessary variables to train the DeepAR model, we can structure the data into separate dataframes for each shiftable device within the household with the **'create_deepar_dataset'** function. 

In [18]:
def create_deepar_dataset(self) -> Dict[str, pd.DataFrame]:
    import numpy as np
    import pandas as pd

    deepar_dataset = {}

    for dev in self.devices.keys():
        hh = self.devices[dev]['hh']
        dev_name = self.devices[dev]['dev_name']

        # Create a DataFrame with required columns
        deepar_dataset[dev] = pd.DataFrame({
            'temp': self.activity_dict[hh]['temp'],
            'dwpt': self.activity_dict[hh]['dwpt'],
            'rhum': self.activity_dict[hh]['rhum'],
            'wdir': self.activity_dict[hh]['wdir'],
            'wspd': self.activity_dict[hh]['wspd'],
            'usage': self.load_dict[hh][dev_name],
            'usage_bin': np.where(self.load_dict[hh][dev_name] > 0, 1, 0),
            'hh': hh,
            'dev': self.devices[dev]['dev'],
            'activity_prob': self.create_activity_probability(self.activity_dict[hh])
        })

        # add usage-related features
        columns = ['periods_since_last_activity', f'periods_since_last_{dev_name}_usage']
        daily_features = self.usage_dict[hh][columns].copy()
        daily_features.columns = daily_features.columns.str.replace(f'{dev_name}_', '', regex=False)
        daily_features['usage_prob'] = self.create_usage_probability(self.usage_dict[hh].copy(), dev_name)

        # Resample daily data to hourly and concatenate
        hourly_features = self.resample_daily_to_hourly_data(daily_features)
        deepar_dataset[dev] = pd.concat([deepar_dataset[dev], hourly_features], axis=1)

        # Select final columns
        columns = [
            'usage', 'usage_bin', 'hh', 'dev',
            'periods_since_last_activity', 'periods_since_last_usage',
            'activity_prob', 'usage_prob',
            'temp', 'dwpt', 'rhum', 'wdir', 'wspd'
        ]

        deepar_dataset[dev] = deepar_dataset[dev][columns]

    return deepar_dataset

setattr(Create_Dataset, 'create_deepar_dataset', create_deepar_dataset)
del create_deepar_dataset


### 2.5 Pipeline

In [19]:
def pipeline(self) -> Dict[str, pd.DataFrame]:
    
    self.slice_datasets()
    
    if self.fill_na:
            self.fill_na_function()

    deepar_dataset = self.create_deepar_dataset()

    return deepar_dataset

setattr(Create_Dataset, 'pipeline', pipeline)
del pipeline



In [20]:
dataset_dict = create_dataset.pipeline()
dataset_dict[0].head(3)


,usage,usage_bin,hh,dev,periods_since_last_activity,periods_since_last_usage,activity_prob,usage_prob,temp,dwpt,rhum,wdir,wspd
2014-03-01 00:00:00,0.0,0,hh1,washing_machine,36.0,37.0,0.006815,0.004902,0.819444,0.518519,0.548148,0.571429,0.609412
2014-03-01 01:00:00,0.0,0,hh1,washing_machine,36.0,37.0,0.007854,0.004902,0.819444,0.518519,0.548148,0.571429,0.609412
2014-03-01 02:00:00,0.0,0,hh1,washing_machine,36.0,37.0,0.005902,0.004902,0.819444,0.518519,0.548148,0.571429,0.609412


## 3. Data Transformation

After preparing the data and shaping it into individual dataframes, our attention turns towards transforming them into a format compatible with GluonTS, a powerful time series forecasting library that includes DeepAR as one of its models. This transformation involves partitioning the data into test, validation, and training sets. We will create these datasets in ListDataset format used by Gluonts, where each time series is represented as a dictionary containing the target values and related features.

The structure of the **'Transform_Dataset'** class is inspired from the following GluonTS template: https://www.kaggle.com/code/steverab/m5-forecasting-competition-gluonts-template

### 3.1 Initialize the class

In a first step, we initialize the **'Transform_Dataset'** class with the necessary parameters.

In [21]:
start_validation = '2015-01-01'
freq = '1H'
prediction_length = 24 #in freq granularity

In [22]:
class Transform_Dataset:
    '''
    A class for reshaping datasets into a format compatible with GluonTS, the forecasting model.
    
    Parameters:
    ----------
    start_training:
        A string ('yyyy-mm-dd') determining the start of the training period.
    start_validation:
        A string ('yyyy-mm-dd') determining the start of the testing period.
    devices:
        A dictionary containing device-specific information.
    dataset_dict:
        A dictionary with target time series data along with covariates.
    freq:
        A string specifying the frequency of the time series data (e.g., '1H' for hourly data).
    prediction_length:
        An integer representing the desired prediction horizon (default is 24).
    '''
    def __init__(
        self,
        start_training: str,
        start_validation: str,
        devices: Dict[str, Any],
        dataset_dict: Dict[str, pd.DataFrame],
        freq: str = '1H',
        prediction_length: int = 24,
    ):
        self.start_training = str(start_training)
        self.start_validation = str(start_validation)
        self.hh_list: List[str] = []
        self.devices = devices
        self.dataset_dict = {hh: df.copy() for hh, df in dataset_dict.items()}
        self.freq = freq
        self.prediction_length = prediction_length


In [23]:
transformer = Transform_Dataset(start_dataset, start_validation, devices, dataset_dict, freq, prediction_length)

### 3.2 Reshape the Data 

The **'reshape_training_data'** method creates a single dataframe where each row represents a device and the columns represent the usage history. Furthermore we add three more columns with the relevant static variables, namely the household id, device id, and the devices' unique identifier. 

In [24]:
def reshape_training_data(self):
    import pandas as pd
    import numpy as np

    transposed_list = {}
    
    for dev in self.devices.keys():
        
        self.dataset_dict[dev] = self.dataset_dict[dev].loc[:self.start_validation]
        usage_bin_df = self.dataset_dict[dev]['usage_bin']
        usage_bin_df = usage_bin_df.reindex()
        usage_bin_df = pd.DataFrame(usage_bin_df)
        usage_bin_df.index = ['d' + str(n) for n in np.arange(usage_bin_df.shape[0])]
        usage_bin_df = pd.DataFrame(usage_bin_df).T

        usage_bin_df['dev'] = self.devices[dev]['dev']
        usage_bin_df['hh'] = str(self.devices[dev]['hh'])
        usage_bin_df['id'] = usage_bin_df['hh'] + '_' + usage_bin_df['dev']
        transposed_list[dev] = usage_bin_df

    training_dataset = pd.concat(transposed_list.values()).reset_index(drop=True)

    return training_dataset

setattr(Transform_Dataset, 'reshape_training_data', reshape_training_data)
del reshape_training_data

In [25]:
training_dataset = transformer.reshape_training_data()
training_dataset.head(3)


,d0,d1,d2,d3,d4,d5,d6,d7,d8,d9,...,d7361,d7362,d7363,d7364,d7365,d7366,d7367,dev,hh,id
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,washing_machine,hh1,hh1_washing_machine
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,dishwasher,hh1,hh1_dishwasher
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,washing_machine,hh2,hh2_washing_machine


### 3.3 Create dynamic features list

Next, we generate dynamic real features for training, validation, and testing using the **'create_dynamic_real_features'** method. These features encompass time-varying continuous variables that describe the activity and usage patterns, as well as the weather conditions. DeepAR leverages these dynamic contextual factors to better capture the underlying patterns within the data.

The dynamic variables are extracted and transposed to a numpy array. For the training and validation lists, we crop the input sequences by removing the last time steps (of prediction lenght) to create a clear separation between the training data and the prediction horizon. This guarantees that the model does not have access to the actual target values it needs to predict during training and validation, thereby preventing data leakage and ensuring that the model is evaluated on unseen data.

In [26]:
def create_dynamic_real_features(self):
    import pandas as pd
    import numpy as np

    train_dynamic_real_features_list = []
    val_dynamic_real_features_list = []
    test_dynamic_real_features_list = []

    for dev in self.devices.keys():
        dynamic_real_columns = self.dataset_dict[dev].drop(columns=['usage','usage_bin','hh','dev']).columns

        train_dynamic_real_features = (
            self.dataset_dict[dev]
            .iloc[: -self.prediction_length * 2, :][dynamic_real_columns]
            .T.to_numpy()
        )
        val_dynamic_real_features = (
            self.dataset_dict[dev]
            .iloc[:-self.prediction_length, :][dynamic_real_columns]
            .T.to_numpy()
        )
        test_dynamic_real_features = self.dataset_dict[dev][
            dynamic_real_columns
        ].T.to_numpy()
        
        train_dynamic_real_features_list.append(train_dynamic_real_features)
        val_dynamic_real_features_list.append(val_dynamic_real_features)
        test_dynamic_real_features_list.append(test_dynamic_real_features)

    return (
        train_dynamic_real_features_list,
        val_dynamic_real_features_list,
        test_dynamic_real_features_list,
    )

setattr(Transform_Dataset, 'create_dynamic_real_features', create_dynamic_real_features)
del create_dynamic_real_features


In [27]:
(train_dynamic_real_features_list,
 val_dynamic_real_features_list,
 test_dynamic_real_features_list,
    ) = transformer.create_dynamic_real_features()
print(len(train_dynamic_real_features_list))
train_dynamic_real_features_list[0].shape

20


(9, 7320)

In [28]:
(train_dynamic_real_features_list,
 val_dynamic_real_features_list,
 test_dynamic_real_features_list,
    ) = transformer.create_dynamic_real_features()
print(len(train_dynamic_real_features_list))
train_dynamic_real_features_list[0].shape


20


(9, 7320)

### 3.4 Create static features list

Next, we define the **'create_static_categorical_features'** function to generate static categorical variables. The function outputs for each device the unique household id and the device type. These values remain constant throughout the time series, and don't need to be repeated along each time step.

In [29]:
def create_static_categorical_features(self, training_dataset: pd.DataFrame) -> Tuple[Any, Tuple[int, int]]:
    import numpy as np
    
    hh_ids = training_dataset['hh'].astype('category').cat.codes.values
    hh_ids_unique = np.unique(hh_ids)

    dev_ids = training_dataset['dev'].astype('category').cat.codes.values
    dev_ids_unique = np.unique(dev_ids)

    stat_cat_list = [hh_ids, dev_ids]

    stat_cat = np.concatenate(stat_cat_list)
    stat_cat = stat_cat.reshape(len(stat_cat_list), len(dev_ids)).T
        
    stat_cat_cardinalities: Tuple[int, int] = (len(hh_ids_unique), len(dev_ids_unique))
        
    return stat_cat, stat_cat_cardinalities

setattr(Transform_Dataset, 'create_static_categorical_features', create_static_categorical_features)
del create_static_categorical_features


In [30]:
stat_cat_cardinalities = transformer.create_static_categorical_features(training_dataset)[1]
stat_cat = transformer.create_static_categorical_features(training_dataset)[0]

print('stat_cat_cardinalities:', stat_cat_cardinalities)
stat_cat


stat_cat_cardinalities: (10, 4)


array([[0, 3],
       [0, 0],
       [2, 3],
       [2, 0],
       [3, 1],
       [3, 3],
       [3, 0],
       [4, 3],
       [4, 3],
       [5, 1],
       [6, 3],
       [6, 0],
       [7, 1],
       [7, 3],
       [7, 0],
       [8, 3],
       [9, 2],
       [9, 3],
       [9, 0],
       [1, 3]], dtype=int8)

### 3.5 Split the data into training, validation, and test

Finally we split the preprocessed data along the covariates into training, validation, and test datasets using the GluonTS library. From the training_dataset we extract the target values for each device usage and create lists of target values for training, validation and testing by cropping them according to the prediction length. Additionally, we create the date list based on the specified start time. Finally, we construct the ListDataset objects to store the time series data along with associated features.

After running this function, we get for each device to be predicted a dictionary containing the target values, the start time of the time series, the dynamic real features, and the static categorical features. This format aligns with the DeepAR model's requirements for effective training and evaluation.

In [31]:
def split_data(
    self,
    training_dataset: pd.DataFrame,
    train_dynamic_real_features_list: List[Any],
    val_dynamic_real_features_list: List[Any],
    test_dynamic_real_features_list: List[Any],
    stat_cat: Any
) -> Tuple[Any, Any, Any]:
    import pandas as pd
    from gluonts.dataset.common import ListDataset
    from gluonts.dataset.field_names import FieldName

    train_df = training_dataset.drop(['id', 'hh', 'dev'], axis=1)
    train_target_values = train_df.values

    test_target_values = val_target_values = train_target_values.copy()

    train_target_values = [ts[:-self.prediction_length * 2] for ts in train_target_values]
    val_target_values = [ts[:-self.prediction_length] for ts in val_target_values]

    dates = [pd.Timestamp(self.start_training) for _ in range(len(training_dataset))]

    def create_list_dataset(target_values, dynamic_real_features_list):
        return ListDataset(
            [
                {
                    FieldName.TARGET: target,
                    FieldName.START: start,
                    FieldName.FEAT_DYNAMIC_REAL: fdr,
                    FieldName.FEAT_STATIC_CAT: fsc,
                }
                for (target, start, fdr, fsc) in zip(
                    target_values,
                    dates,
                    dynamic_real_features_list,
                    stat_cat,
                )
            ],
            freq=self.freq,
        )

    train_ds = create_list_dataset(train_target_values, train_dynamic_real_features_list)
    val_ds = create_list_dataset(val_target_values, val_dynamic_real_features_list)
    test_ds = create_list_dataset(test_target_values, test_dynamic_real_features_list)

    return train_ds, val_ds, test_ds

setattr(Transform_Dataset, 'split_data', split_data)
del split_data


In [32]:
train_ds, _, _, = transformer.split_data(training_dataset,
                                          train_dynamic_real_features_list,
                                          val_dynamic_real_features_list,
                                          test_dynamic_real_features_list,
                                          stat_cat
                                         )
train_ds[0]['feat_dynamic_real'].shape


(9, 7320)

### 3.6 Pipeline

Finally, the pipeline function streamlines the process and generates training, validation, and test datasets for training and evaluating the DeepAR model.

In [33]:
def pipeline(self) -> Tuple[Any, Any, Any]:
    import pandas as pd
    import numpy as np
    from gluonts.dataset.common import ListDataset
    from gluonts.dataset.field_names import FieldName

    training_dataset = self.reshape_training_data()

    (
        train_dynamic_real_features_list,
        val_dynamic_real_features_list,
        test_dynamic_real_features_list,
    ) = self.create_dynamic_real_features()
        
    stat_cat = self.create_static_categorical_features(training_dataset)[0]

    train_ds, val_ds, test_ds = self.split_data(
        training_dataset,
        train_dynamic_real_features_list,
        val_dynamic_real_features_list,
        test_dynamic_real_features_list,
        stat_cat
    )

    return train_ds, val_ds, test_ds

setattr(Transform_Dataset, 'pipeline', pipeline)
del pipeline


In [34]:
_, _, test_ds, = transformer.pipeline()
test_ds[0]


{'target': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 'start': Period('2014-03-01 00:00', 'H'),
 'feat_dynamic_real': array([[3.6000000e+01, 3.6000000e+01, 3.6000000e+01, ..., 1.0000000e+00,
         1.0000000e+00, 1.0000000e+00],
        [3.7000000e+01, 3.7000000e+01, 3.7000000e+01, ..., 2.0000000e+00,
         2.0000000e+00, 2.0000000e+00],
        [6.8152905e-03, 7.8541255e-03, 5.9017446e-03, ..., 9.9433905e-01,
         9.8637235e-01, 9.9727309e-01],
        ...,
        [5.4814816e-01, 5.4814816e-01, 5.4814816e-01, ..., 6.6666669e-01,
         6.6666669e-01, 6.6666669e-01],
        [5.7142860e-01, 5.7142860e-01, 5.7142860e-01, ..., 5.7142860e-01,
         5.7142860e-01, 5.7142860e-01],
        [6.0941178e-01, 6.0941178e-01, 6.0941178e-01, ..., 0.0000000e+00,
         1.0000000e+00, 9.1529411e-01]], dtype=float32),
 'feat_static_cat': array([0, 3], dtype=int32)}

## 4. Model Training

With our datasets prepared, we are now ready to initiate the hyperparameter search, aiming to discover the optimal model configuration for the forecasting task.

### 4.1 Early stopping

Conducting hyperparameter search for a DeepAR model can be time-sensitive. To efficiently manage this, we implement a custom callback that can terminate the training process early based on a specified metric value. The class **'Metric_Inference_Early_Stopping'** is taken from the GluonTS website, where they introduce callbacks to the Trainer class.

Link: https://ts.gluon.ai/dev/tutorials/advanced_topics/trainer_callbacks.html

In [35]:
from gluonts.mx.trainer.callback import Callback
from gluonts.evaluation import Evaluator
from gluonts.dataset.common import Dataset
from gluonts.mx import DeepAREstimator # Trainer

class Metric_Inference_Early_Stopping(Callback):
    '''
    Early Stopping mechanism based on the prediction network.
    Can be used to base the Early Stopping directly on a metric of interest, instead of on the training/validation loss.
    In the same way as test datasets are used during model evaluation,
    the time series of the validation_dataset can overlap with the train dataset time series,
    except for a prediction_length part at the end of each time series.
    Parameters
    ----------
    validation_dataset
        An out-of-sample dataset which is used to monitor metrics
    predictor
        A gluon predictor, with a prediction network that matches the training network
    evaluator
        The Evaluator used to calculate the validation metrics.
    metric
        The metric on which to base the early stopping on.
    patience
        Number of epochs to train on given the metric did not improve more than min_delta.
    min_delta
        Minimum change in the monitored metric counting as an improvement
    verbose
        Controls, if the validation metric is printed after each epoch.
    minimize_metric
        The metric objective.
    restore_best_network
        Controls, if the best model, as assessed by the validation metrics is restored after training.
    num_samples
        The amount of samples drawn to calculate the inference metrics.
    '''

    def __init__(
        self,
        validation_dataset: Dataset,
        estimator: DeepAREstimator,
        evaluator: Evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9],allow_nan_forecast=True),
        metric: str = 'MSE',
        patience: int = 20,
        min_delta: float = 0.0,
        verbose: bool = False,
        minimize_metric: bool = True,
        restore_best_network: bool = True,
        num_samples: int = 100,
    ):
        assert (
            patience >= 0
        ), 'EarlyStopping Callback patience needs to be >= 0'
        assert (
            min_delta >= 0
        ), 'EarlyStopping Callback min_delta needs to be >= 0.0'
        assert (
            num_samples >= 1
        ), 'EarlyStopping Callback num_samples needs to be >= 1'

        self.validation_dataset = list(validation_dataset)
        self.estimator = estimator
        self.evaluator = evaluator
        self.metric = metric
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        self.restore_best_network = restore_best_network
        self.num_samples = num_samples

        if minimize_metric:
            self.best_metric_value = np.inf
            self.is_better = np.less
        else:
            self.best_metric_value = -np.inf
            self.is_better = np.greater

        self.validation_metric_history: List[float] = []
        self.best_network = None
        self.n_stale_epochs = 0

    def on_epoch_end(
        self,
        epoch_no: int,
        epoch_loss: float,
        training_network: mx.gluon.nn.HybridBlock,
        trainer: mx.gluon.Trainer,
        best_epoch_info: dict,
        ctx: mx.Context
    ) -> bool:
        should_continue = True
        
        transformation = self.estimator.create_transformation()
        predictor = self.estimator.create_predictor(transformation=transformation, trained_network=training_network)

        from gluonts.evaluation.backtest import make_evaluation_predictions

        forecast_it, ts_it = make_evaluation_predictions(
            dataset=self.validation_dataset,
            predictor=predictor,
            num_samples=self.num_samples,
        )

        agg_metrics, item_metrics = self.evaluator(ts_it, forecast_it)
        current_metric_value = agg_metrics[self.metric]
        self.validation_metric_history.append(current_metric_value)

        if self.verbose:
            print(
                f'Validation metric {self.metric}: {current_metric_value}, best: {self.best_metric_value}'
            )

        if self.is_better(current_metric_value, self.best_metric_value):
            self.best_metric_value = current_metric_value

            if self.restore_best_network:
                training_network.save_parameters('best_network.params')

            self.n_stale_epochs = 0
        else:
            self.n_stale_epochs += 1
            if self.n_stale_epochs == self.patience:
                should_continue = False
                print(
                    f'EarlyStopping callback initiated stop of training at epoch {epoch_no}.'
                )

                if self.restore_best_network:
                    print(
                        f'Restoring best network from epoch {epoch_no - self.patience}.'
                    )
                    training_network.load_parameters('best_network.params')

        return should_continue
    

### 4.2 Hyperparameter Tuning

We will tune the hyperparameters of the DeepAR model using Optuna, an automated hyperparameter optimization framework that explores the search space to find optimal configurations. The initial class created for model tuning can be found on the GluonTS website: https://ts.gluon.ai/dev/tutorials/advanced_topics/hp_tuning_with_optuna.html

However, we make small changes to this class so that it fits our task of evaluating the model over a longer period of time.

#### 4.2.1 Initialize the Class

In [36]:
recommendation_length = 31

In [37]:
from gluonts.mx.distribution import CategoricalOutput
from gluonts.mx import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions

class DeepAR_Tuning_Objective:
    '''
    A class for hyperparameter tuning using Optuna.

    Parameters:
    ----------
    prediction_length:
        An integer representing the desired prediction horizon.
    freq:
        A string specifying the frequency of the time series data.
    start_training:
        A string ('yyyy-mm-dd') determining the start of the training period.
    start_validation:
        A string ('yyyy-mm-dd') determining the start of the validation period.
    recommendation_length:
        The length of the test period in days.
    devices:
        A dictionary containing device-specific information.
    dataset_dict:
        A dictionary with target time series data along with covariates.
    metric_type:
        A string specifying the type of metric to optimize during tuning (default is 'MASE').
    '''
    def __init__(
        self,
        prediction_length: int,
        freq: str,
        start_training: str,
        start_validation: str,
        recommendation_length: int,
        devices: Dict[str, Any],
        dataset_dict: Dict[str, pd.DataFrame],
        metric_type='MASE', 
    ):
        self.prediction_length = prediction_length
        self.freq = freq
        self.start_training = start_training
        self.start_validation = start_validation
        self.recommendation_length = recommendation_length
        self.devices = devices
        self.dataset_dict = dataset_dict
        self.metric_type = metric_type


In [38]:
DeepAR_tuning = DeepAR_Tuning_Objective(prediction_length, 
                                        freq,
                                        start_dataset,
                                        start_validation,
                                        recommendation_length,
                                        devices,
                                        dataset_dict)


#### 4.2.2 Define the Hyperparameters

With the **'get_params'** function, we define what hyperparameters to be tuned. These hyperparameters are sampled within specified ranges during the optimization process.

In [39]:
def get_params(self, trial) -> dict:
    return {
        # Number of time steps in the context window (3, 4, or 5 weeks)
        'context_length': trial.suggest_categorical('context_length', [504, 672, 840]),
        
        # Number of layers in the network
        'num_layers': trial.suggest_int('num_layers', 3, 4),
        
        # Number of cells in each layer
        'num_cells': trial.suggest_int('num_cells', 10, 50),
        
        # Dropout rate to prevent overfitting
        'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.5),
        
        # Learning rate for optimization (log scale)
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True),
        
        # Number of training epochs
        'epochs': trial.suggest_int('epochs', 25, 100),
        
        # Batch size for training
        'batch_size': trial.suggest_categorical('batch_size', [32, 64, 128]),
        
        # Number of batches per epoch
        'num_batches_per_epoch': trial.suggest_categorical('num_batches_per_epoch', [40, 50, 60, 70, 80, 90]),
    }

setattr(DeepAR_Tuning_Objective, 'get_params', get_params)
del get_params


#### 4.2.3 Hyperparameter Tuning and Evaluation 


In the **'\_call_'** function, we set up the DeepAR model for training and evaluation. Here's a step-by-step breakdown of the process:

1. Hyperparameter Setup:
We obtain a set of hyperparameters using the **'get_params'** method. Optuna provides suggestions for these hyperparameters during each trial. With these hyperparameters, we configure an instance of the DeepAR model. The estimator is set up with other relevant configuration parameters, like the prediction length, the frequency of the time series, output distribution (CategoricalOutput), and the use of dynamic and static features.

2. Evaluator Configuration:
We establish an Evaluator function which will help us assess how well the model performs.

3. Data Preparation for Training and Evaluation:
To train and evaluate the model, we begin by creating a date list, which contains the days for which we wish to make predictions. Subsequently, we generate training and evaluation datasets with **'TransformDataset'** instance.

4. Early Stopping and Trainer Configuration:
We integrate the early stopping callback to track the model's performance on the validation dataset. If the xxx metric doesn't improve for a defined number of epochs, the training process halts. We then configure the trainer settings, including the early stopping callback. The trainer is responsible for managing the training process.

5. Model Training:
We initiate the training of the DeepAR model using the training dataset train_ds. The model learns from the data to make predictions.

6. Evaluation and Error Computation:
To evaluate this model, we make predictions using the trained predictor for each date in 'date_list_daily'. Utilizing the make_evaluation_predictions method, we obtain forecasts and true time series (forecasts, tss). We calculate binary forecasts based on the median sample from the forecasts dataset, compare them with true values, and compute the daily error.

7. Optuna Trial Update and Error Return:
We update the Optuna trial's user attribute with the best trained model for future reference and return the cumulative daily_error, which signifies the total difference between the binary forecasts and the true values.

In [40]:
def __call__(self, trial):
    params = self.get_params(trial)

    transformer = Transform_Dataset(
        self.start_training,
        self.start_validation,
        self.devices,
        self.dataset_dict,
        self.freq, 
        self.prediction_length)
    
    training_dataset = transformer.reshape_training_data()
    
    # Create static categorical features
    cardinality = transformer.create_static_categorical_features(training_dataset)[1]

    # Initialize the DeepAR estimator
    estimator = DeepAREstimator(
        prediction_length=self.prediction_length,
        context_length=params['context_length'],
        freq=self.freq,
        distr_output=CategoricalOutput(2),
        use_feat_dynamic_real=True,
        use_feat_static_cat=True,
        cardinality=cardinality,
        num_layers=params['num_layers'],
        num_cells=params['num_cells'],
        batch_size=params['batch_size'],
        dropout_rate=params['dropout_rate'],
    )
    
    evaluator = Evaluator(allow_nan_forecast=True)

    train_ds, val_ds, _ = transformer.pipeline()
    
    es_callback = Metric_Inference_Early_Stopping(
        validation_dataset=val_ds,
        estimator=estimator,
        metric='RMSE',
        patience=20,
        evaluator=evaluator
    )

    trainer = Trainer(
        learning_rate=params['learning_rate'],
        epochs=params['epochs'],
        num_batches_per_epoch=params['num_batches_per_epoch'],
        callbacks=[es_callback],
        hybridize=False
    )
    
    datelist_daily = pd.date_range(self.start_validation, freq='d', periods=self.recommendation_length)

    estimator.trainer = trainer
    global predictor
    daily_error = 0
    

    predictor = estimator.train(train_ds)

    for date in datelist_daily:
        
        transformer = Transform_Dataset(
            self.start_training,
            date,
            self.devices,
            self.dataset_dict,
            self.freq, 
            self.prediction_length)

        _, _, test_ds = transformer.pipeline()

        forecast_deepar, ts_deepar = make_evaluation_predictions(
            dataset=test_ds,
            predictor=predictor,
            num_samples=100
        )
        forecasts = list(forecast_deepar)
        tss = list(ts_deepar)

        forecasts_bin = {}
        for i in range(0, len(forecasts)):

            forecasts_bin[i] = np.where(forecasts[i].quantile(0.5) >= 0.5, 1, 0)
            daily_error += abs(np.where(forecasts_bin[i].sum() == 0, 0, 1) - np.where(tss[i][-24:].sum()[0] == 0, 0, 1))
            
    trial.set_user_attr(key='best_model', value=predictor)

    return daily_error

setattr(DeepAR_Tuning_Objective, '__call__', __call__)
del __call__

#### 4.2.4 Find the Best  Hyperparameters with Optuna

Finally we employ Optuna in the **'optimize_deepar_hyperparameters'** function. A study is initiated to minimize the objective function. Using the **'DeepAR_Tuning_Objective'** class, the objective encapsulates model training and evaluation. The optimization process searches for the best hyperparameters over a specified number of trials. and with the callback function, tracks the best trial, updating best_model_deepar when a better model is found.

In [41]:
def optimize_deepar_hyperparameters(
    prediction_length, freq, start_dataset, start_validation, recommendation_length, devices, dataset_dict, n_trials=30, seed=0
):
    import optuna
    import random
    import numpy as np
    import time

    random.seed(seed)
    np.random.seed(seed)

    best_model_deepar = None
    predictor = None

    def callback(study, trial):
        nonlocal best_model_deepar
        if study.best_trial == trial:
            best_model_deepar = predictor

    time_start = time.time()
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=seed))
    study.optimize(
        DeepAR_Tuning_Objective(
            prediction_length, freq, start_dataset, start_validation, recommendation_length, devices, dataset_dict
        ),
        n_trials=n_trials,
        callbacks=[callback]
    )

    print('Number of finished trials: {}'.format(len(study.trials)))

    print('Best trial:')
    trial_p = study.best_trial

    print('  Value: {}'.format(trial_p.value))

    print('  Params: ')
    for key, value in trial_p.params.items():
        print('    {}: {}'.format(key, value))
    print('Elapsed time:', time.time() - time_start)

    return best_model_deepar


In the cell below, you can upload the pickle file for the best model. We will evaluate it on the next script. This DeepAR model has the following parameters:

* Context length: 840
* No of cells: 48
* No of layers: 3
* Learning rate: 0.01752
* Dropout rate: 0.16056
* Epocs: 31
* Batch size: 128
* No of batches per epoch: 80

In [42]:
best_model = Helper_Functions_Thesis.open_pickle_file(PICKLE_PATH,'deepar_model.pickle')

<br>
<br>
<br>

## **Appendix A1: Complete Create_Dataset Class**

In [43]:

class Create_Dataset:
    '''
    A class for constructing datasets for each device with all the necessary information to train
    the forecasting model.
    Parameters:
    ----------
    start_dataset:
        A string ('yyyy-mm-dd') to determine the start of the dataset.
    end_dataset:
        A string ('yyyy-mm-dd') to determine the end of the dataset.
    devices:
        A dictionary containing device-specific information.
    load_dict:
        A dictionary containing the output dataframes from the Load Agent for each household.
    usage_dict:
        A dictionary containing the output dataframes from the Usage Agent for each household.
    activity_dict:
        A dictionary containing the output dataframes from the Activity Agent for each household.
    fill_na:
        Bool, controls if missing values should be filled with zeros.
    '''

    def __init__(
        self,
        start_dataset: str,
        end_dataset: str,
        devices: Dict[str, Any],
        load_dict: Dict[str, pd.DataFrame],
        usage_dict: Dict[str, pd.DataFrame],
        activity_dict: Dict[str, pd.DataFrame],
        fill_na = True,
    ):
        from helper_functions_thesis import Helper_Functions_Thesis
        self.start_dataset = start_dataset
        self.end_dataset = end_dataset
        self.devices = devices
        self.load_dict = load_dict
        self.usage_dict = usage_dict
        self.activity_dict = activity_dict
        self.hh_list: List[str] = list(Helper_Functions_Thesis.create_shiftable_devices_dict(self.devices).keys())
        self.fill_na = fill_na

    def slice_datasets(self) -> None:
        import pandas as pd

        for hh in self.hh_list:
            self.load_dict[hh] = self.load_dict[hh].loc[self.start_dataset:self.end_dataset, :]
            self.usage_dict[hh] = self.usage_dict[hh].loc[self.start_dataset:self.end_dataset, :]
            self.activity_dict[hh] = self.activity_dict[hh].loc[self.start_dataset:self.end_dataset, :]

    def fill_na_function(self) -> None:
        import pandas as pd

        for hh in self.hh_list:
            self.load_dict[hh] = self.load_dict[hh].fillna(0)
            self.usage_dict[hh] = self.usage_dict[hh].fillna(0)
            self.activity_dict[hh] = self.activity_dict[hh].fillna(0)

    def create_activity_probability(self, activity_dict: pd.DataFrame) -> np.ndarray:
        import numpy as np
        from datetime import timedelta
        from agents import Activity_Agent
            
        activity = Activity_Agent(activity_dict)
        X_train, y_train, X_test, y_test = activity.train_test_split(
            activity_dict, str(activity_dict.index[-1].date()+timedelta(1))
        )
        X_train = X_train.fillna(0)
        model = activity.fit(X_train, y_train, 'random forest')
        activity_probability = model.predict_proba(X_train)[:, 1]
        return activity_probability

    def create_usage_probability(self, usage_dict: pd.DataFrame, dev_name: str) -> np.ndarray:
        import numpy as np
        from agents import Usage_Agent
        
        usage = Usage_Agent(usage_dict, dev_name)
        X_train, y_train, X_test, y_test = usage.train_test_split(
            usage_dict, self.end_dataset, train_start=self.start_dataset
        )
        X_train = X_train.fillna(0) 
        model = usage.fit(X_train, y_train.values.ravel(), 'random forest')
        usage_probability = model.predict_proba(X_train)[:, 1]
        return usage_probability

    def resample_daily_to_hourly_data(self, daily_data: pd.DataFrame) -> pd.DataFrame:
        import pandas as pd
        from agents import Activity_Agent

        dummy_day = daily_data.index[-1] + pd.Timedelta(days=1)
        dummy_row = pd.DataFrame(index=pd.DatetimeIndex([dummy_day]))
        daily_data_with_dummy = pd.concat([daily_data, dummy_row])
        hourly_data = daily_data_with_dummy.resample("H").ffill().iloc[:-1, :]
        return hourly_data

    def create_deepar_dataset(self) -> Dict[str, pd.DataFrame]:
        import numpy as np
        import pandas as pd

        deepar_dataset = {}

        for dev in self.devices.keys():
            hh = self.devices[dev]['hh']
            dev_name = self.devices[dev]['dev_name']

            # Create a DataFrame with required columns
            deepar_dataset[dev] = pd.DataFrame({
                'temp': self.activity_dict[hh]['temp'],
                'dwpt': self.activity_dict[hh]['dwpt'],
                'rhum': self.activity_dict[hh]['rhum'],
                'wdir': self.activity_dict[hh]['wdir'],
                'wspd': self.activity_dict[hh]['wspd'],
                'usage': self.load_dict[hh][dev_name],
                'usage_bin': np.where(self.load_dict[hh][dev_name] > 0, 1, 0),
                'hh': hh,
                'dev': self.devices[dev]['dev'],
                'activity_prob': self.create_activity_probability(self.activity_dict[hh])
            })

            # add usage-related features
            columns = ['periods_since_last_activity', f'periods_since_last_{dev_name}_usage']
            daily_features = self.usage_dict[hh][columns].copy()
            daily_features.columns = daily_features.columns.str.replace(f'{dev_name}_', '', regex=False)
            daily_features['usage_prob'] = self.create_usage_probability(self.usage_dict[hh].copy(), dev_name)

            # Resample daily data to hourly and concatenate
            hourly_features = self.resample_daily_to_hourly_data(daily_features)
            deepar_dataset[dev] = pd.concat([deepar_dataset[dev], hourly_features], axis=1)

            # Select final columns
            columns = [
                'usage', 'usage_bin', 'hh', 'dev',
                'periods_since_last_activity', 'periods_since_last_usage',
                'activity_prob', 'usage_prob',
                'temp', 'dwpt', 'rhum', 'wdir', 'wspd'
            ]

            deepar_dataset[dev] = deepar_dataset[dev][columns]

        return deepar_dataset



    def pipeline(self) -> Dict[str, pd.DataFrame]:
    
        self.slice_datasets()
        
        if self.fill_na:
                self.fill_na_function()

        deepar_dataset = self.create_deepar_dataset()

        return deepar_dataset


## **Appendix A2: Complete Transform_Dataset Class**

In [44]:
from typing import Dict, Any, List, Tuple
import pandas as pd
class Transform_Dataset:
    '''
    A class for reshaping datasets into a format compatible with GluonTS, the forecasting model.
    
    Parameters:
    ----------
    start_training:
        A string ('yyyy-mm-dd') determining the start of the training period.
    start_validation:
        A string ('yyyy-mm-dd') determining the start of the testing period.
    devices:
        A dictionary containing device-specific information.
    dataset_dict:
        A dictionary with target time series data along with covariates.
    freq:
        A string specifying the frequency of the time series data (e.g., '1H' for hourly data).
    prediction_length:
        An integer representing the desired prediction horizon (default is 24).
    '''
    def __init__(
        self,
        start_training: str,
        start_validation: str,
        devices: Dict[str, Any],
        dataset_dict: Dict[str, pd.DataFrame],
        freq: str = '1H',
        prediction_length: int = 24,
    ):
        self.start_training = str(start_training)
        self.start_validation = str(start_validation)
        self.hh_list: List[str] = []
        self.devices = devices
        self.dataset_dict = {hh: df.copy() for hh, df in dataset_dict.items()}
        self.freq = freq
        self.prediction_length = prediction_length
        
    def reshape_training_data(self):
        import pandas as pd
        import numpy as np

        transposed_list = {}
        
        for dev in self.devices.keys():
            
            self.dataset_dict[dev] = self.dataset_dict[dev].loc[:self.start_validation]
            usage_bin_df = self.dataset_dict[dev]['usage_bin']
            usage_bin_df = usage_bin_df.reindex()
            usage_bin_df = pd.DataFrame(usage_bin_df)
            usage_bin_df.index = ['d' + str(n) for n in np.arange(usage_bin_df.shape[0])]
            usage_bin_df = pd.DataFrame(usage_bin_df).T

            usage_bin_df['dev'] = self.devices[dev]['dev']
            usage_bin_df['hh'] = str(self.devices[dev]['hh'])
            usage_bin_df['id'] = usage_bin_df['hh'] + '_' + usage_bin_df['dev']
            transposed_list[dev] = usage_bin_df

        training_dataset = pd.concat(transposed_list.values()).reset_index(drop=True)

        return training_dataset

    def create_dynamic_real_features(self):
        import pandas as pd
        import numpy as np

        train_dynamic_real_features_list = []
        val_dynamic_real_features_list = []
        test_dynamic_real_features_list = []

        for dev in self.devices.keys():
            dynamic_real_columns = self.dataset_dict[dev].drop(columns=['usage','usage_bin','hh','dev']).columns

            train_dynamic_real_features = (
                self.dataset_dict[dev]
                .iloc[: -self.prediction_length * 2, :][dynamic_real_columns]
                .T.to_numpy()
            )
            val_dynamic_real_features = (
                self.dataset_dict[dev]
                .iloc[:-self.prediction_length, :][dynamic_real_columns]
                .T.to_numpy()
            )
            test_dynamic_real_features = self.dataset_dict[dev][
                dynamic_real_columns
            ].T.to_numpy()

            train_dynamic_real_features_list.append(train_dynamic_real_features)
            val_dynamic_real_features_list.append(val_dynamic_real_features)
            test_dynamic_real_features_list.append(test_dynamic_real_features)

        return (
            train_dynamic_real_features_list,
            val_dynamic_real_features_list,
            test_dynamic_real_features_list,
        )

    def create_static_categorical_features(self, training_dataset: pd.DataFrame) -> Tuple[Any, Tuple[int, int]]:
        import numpy as np
        
        hh_ids = training_dataset['hh'].astype('category').cat.codes.values
        hh_ids_unique = np.unique(hh_ids)

        dev_ids = training_dataset['dev'].astype('category').cat.codes.values
        dev_ids_unique = np.unique(dev_ids)

        stat_cat_list = [hh_ids, dev_ids]

        stat_cat = np.concatenate(stat_cat_list)
        stat_cat = stat_cat.reshape(len(stat_cat_list), len(dev_ids)).T
            
        stat_cat_cardinalities: Tuple[int, int] = (len(hh_ids_unique), len(dev_ids_unique))
            
        return stat_cat, stat_cat_cardinalities

    def split_data(
    self,
    training_dataset: pd.DataFrame,
    train_dynamic_real_features_list: List[Any],
    val_dynamic_real_features_list: List[Any],
    test_dynamic_real_features_list: List[Any],
    stat_cat: Any
) -> Tuple[Any, Any, Any]:
        import pandas as pd
        from gluonts.dataset.common import ListDataset
        from gluonts.dataset.field_names import FieldName

        train_df = training_dataset.drop(['id', 'hh', 'dev'], axis=1)
        train_target_values = train_df.values

        test_target_values = val_target_values = train_target_values.copy()

        train_target_values = [ts[:-self.prediction_length * 2] for ts in train_target_values]
        val_target_values = [ts[:-self.prediction_length] for ts in val_target_values]

        dates = [pd.Timestamp(self.start_training) for _ in range(len(training_dataset))]

        def create_list_dataset(target_values, dynamic_real_features_list):
            return ListDataset(
                [
                    {
                        FieldName.TARGET: target,
                        FieldName.START: start,
                        FieldName.FEAT_DYNAMIC_REAL: fdr,
                        FieldName.FEAT_STATIC_CAT: fsc,
                    }
                    for (target, start, fdr, fsc) in zip(
                        target_values,
                        dates,
                        dynamic_real_features_list,
                        stat_cat,
                    )
                ],
                freq=self.freq,
            )

        train_ds = create_list_dataset(train_target_values, train_dynamic_real_features_list)
        val_ds = create_list_dataset(val_target_values, val_dynamic_real_features_list)
        test_ds = create_list_dataset(test_target_values, test_dynamic_real_features_list)

        return train_ds, val_ds, test_ds

    def pipeline(self) -> Tuple[Any, Any, Any]:
        import pandas as pd
        import numpy as np
        from gluonts.dataset.common import ListDataset
        from gluonts.dataset.field_names import FieldName

        training_dataset = self.reshape_training_data()

        (
            train_dynamic_real_features_list,
            val_dynamic_real_features_list,
            test_dynamic_real_features_list,
        ) = self.create_dynamic_real_features()
            
        stat_cat = self.create_static_categorical_features(training_dataset)[0]

        train_ds, val_ds, test_ds = self.split_data(
            training_dataset,
            train_dynamic_real_features_list,
            val_dynamic_real_features_list,
            test_dynamic_real_features_list,
            stat_cat
        )

        return train_ds, val_ds, test_ds


## **Appendix A3: Complete Metric_Inference_Early_Stopping Class'**

In [45]:
import mxnet as mx
from gluonts.mx.trainer.callback import Callback
from gluonts.evaluation import Evaluator
from gluonts.dataset.common import Dataset
from gluonts.mx import DeepAREstimator
class Metric_Inference_Early_Stopping(Callback):
    '''
    Early Stopping mechanism based on the prediction network.
    Can be used to base the Early Stopping directly on a metric of interest, instead of on the training/validation loss.
    In the same way as test datasets are used during model evaluation,
    the time series of the validation_dataset can overlap with the train dataset time series,
    except for a prediction_length part at the end of each time series.
    Parameters
    ----------
    validation_dataset
        An out-of-sample dataset which is used to monitor metrics
    predictor
        A gluon predictor, with a prediction network that matches the training network
    evaluator
        The Evaluator used to calculate the validation metrics.
    metric
        The metric on which to base the early stopping on.
    patience
        Number of epochs to train on given the metric did not improve more than min_delta.
    min_delta
        Minimum change in the monitored metric counting as an improvement
    verbose
        Controls, if the validation metric is printed after each epoch.
    minimize_metric
        The metric objective.
    restore_best_network
        Controls, if the best model, as assessed by the validation metrics is restored after training.
    num_samples
        The amount of samples drawn to calculate the inference metrics.
    '''

    def __init__(
        self,
        validation_dataset: Dataset,
        estimator: DeepAREstimator,
        evaluator: Evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9],allow_nan_forecast=True),
        metric: str = 'MSE',
        patience: int = 20,
        min_delta: float = 0.0,
        verbose: bool = False,
        minimize_metric: bool = True,
        restore_best_network: bool = True,
        num_samples: int = 100,
    ):
        assert (
            patience >= 0
        ), 'EarlyStopping Callback patience needs to be >= 0'
        assert (
            min_delta >= 0
        ), 'EarlyStopping Callback min_delta needs to be >= 0.0'
        assert (
            num_samples >= 1
        ), 'EarlyStopping Callback num_samples needs to be >= 1'

        self.validation_dataset = list(validation_dataset)
        self.estimator = estimator
        self.evaluator = evaluator
        self.metric = metric
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        self.restore_best_network = restore_best_network
        self.num_samples = num_samples

        if minimize_metric:
            self.best_metric_value = np.inf
            self.is_better = np.less
        else:
            self.best_metric_value = -np.inf
            self.is_better = np.greater

        self.validation_metric_history: List[float] = []
        self.best_network = None
        self.n_stale_epochs = 0

    def on_epoch_end(
        self,
        epoch_no: int,
        epoch_loss: float,
        training_network: mx.gluon.nn.HybridBlock,
        trainer: mx.gluon.Trainer,
        best_epoch_info: dict,
        ctx: mx.Context
    ) -> bool:
        should_continue = True
        
        transformation = self.estimator.create_transformation()
        predictor = self.estimator.create_predictor(transformation=transformation, trained_network=training_network)

        from gluonts.evaluation.backtest import make_evaluation_predictions

        forecast_it, ts_it = make_evaluation_predictions(
            dataset=self.validation_dataset,
            predictor=predictor,
            num_samples=self.num_samples,
        )

        agg_metrics, item_metrics = self.evaluator(ts_it, forecast_it)
        current_metric_value = agg_metrics[self.metric]
        self.validation_metric_history.append(current_metric_value)

        if self.verbose:
            print(
                f'Validation metric {self.metric}: {current_metric_value}, best: {self.best_metric_value}'
            )

        if self.is_better(current_metric_value, self.best_metric_value):
            self.best_metric_value = current_metric_value

            if self.restore_best_network:
                training_network.save_parameters('best_network.params')

            self.n_stale_epochs = 0
        else:
            self.n_stale_epochs += 1
            if self.n_stale_epochs == self.patience:
                should_continue = False
                print(
                    f'EarlyStopping callback initiated stop of training at epoch {epoch_no}.'
                )

                if self.restore_best_network:
                    print(
                        f'Restoring best network from epoch {epoch_no - self.patience}.'
                    )
                    training_network.load_parameters('best_network.params')

        return should_continue


## **Appendix A4: Complete DeepAR_Tuning_Objective Class'**

In [46]:

class DeepAR_Tuning_Objective:
    '''
    A class for hyperparameter tuning using Optuna.

    Parameters:
    ----------
    prediction_length:
        An integer representing the desired prediction horizon.
    freq:
        A string specifying the frequency of the time series data.
    start_training:
        A string ('yyyy-mm-dd') determining the start of the training period.
    start_validation:
        A string ('yyyy-mm-dd') determining the start of the validation period.
    recommendation_length:
        The length of the test period in days.
    devices:
        A dictionary containing device-specific information.
    dataset_dict:
        A dictionary with target time series data along with covariates.
    metric_type:
        A string specifying the type of metric to optimize during tuning (default is 'MASE').
    '''
    def __init__(
        self,
        prediction_length: int,
        freq: str,
        start_training: str,
        start_validation: str,
        recommendation_length: int,
        devices: Dict[str, Any],
        dataset_dict: Dict[str, pd.DataFrame],
        metric_type='MASE', 
    ):
        self.prediction_length = prediction_length
        self.freq = freq
        self.start_training = start_training
        self.start_validation = start_validation
        self.recommendation_length = recommendation_length
        self.devices = devices
        self.dataset_dict = dataset_dict
        self.metric_type = metric_type
        
    def get_params(self, trial) -> dict:
        return {
            # Number of time steps in the context window (3, 4, or 5 weeks)
            'context_length': trial.suggest_categorical('context_length', [504, 672, 840]),
            
            # Number of layers in the network
            'num_layers': trial.suggest_int('num_layers', 3, 4),
            
            # Number of cells in each layer
            'num_cells': trial.suggest_int('num_cells', 10, 50),
            
            # Dropout rate to prevent overfitting
            'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.5),
            
            # Learning rate for optimization (log scale)
            'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True),
            
            # Number of training epochs
            'epochs': trial.suggest_int('epochs', 25, 100),
            
            # Batch size for training
            'batch_size': trial.suggest_categorical('batch_size', [32, 64, 128]),
            
            # Number of batches per epoch
            'num_batches_per_epoch': trial.suggest_categorical('num_batches_per_epoch', [40, 50, 60, 70, 80, 90]),
        }

    def __call__(self, trial):
        params = self.get_params(trial)

        transformer = Transform_Dataset(
            self.start_training,
            self.start_validation,
            self.devices,
            self.dataset_dict,
            self.freq, 
            self.prediction_length)
        
        training_dataset = transformer.reshape_training_data()
        
        # Create static categorical features
        cardinality = transformer.create_static_categorical_features(training_dataset)[1]

        # Initialize the DeepAR estimator
        estimator = DeepAREstimator(
            prediction_length=self.prediction_length,
            context_length=params['context_length'],
            freq=self.freq,
            distr_output=CategoricalOutput(2),
            use_feat_dynamic_real=True,
            use_feat_static_cat=True,
            cardinality=cardinality,
            num_layers=params['num_layers'],
            num_cells=params['num_cells'],
            batch_size=params['batch_size'],
            dropout_rate=params['dropout_rate'],
        )
        
        evaluator = Evaluator(allow_nan_forecast=True)

        train_ds, val_ds, _ = transformer.pipeline()
        
        es_callback = Metric_Inference_Early_Stopping(
            validation_dataset=val_ds,
            estimator=estimator,
            metric='RMSE',
            patience=20,
            evaluator=evaluator
        )

        trainer = Trainer(
            learning_rate=params['learning_rate'],
            epochs=params['epochs'],
            num_batches_per_epoch=params['num_batches_per_epoch'],
            callbacks=[es_callback],
            hybridize=False
        )
        
        datelist_daily = pd.date_range(self.start_validation, freq='d', periods=self.recommendation_length)

        estimator.trainer = trainer
        global predictor
        daily_error = 0
        

        predictor = estimator.train(train_ds)

        for date in datelist_daily:
            
            transformer = Transform_Dataset(
                self.start_training,
                date,
                self.devices,
                self.dataset_dict,
                self.freq, 
                self.prediction_length)

            _, _, test_ds = transformer.pipeline()

            forecast_deepar, ts_deepar = make_evaluation_predictions(
                dataset=test_ds,
                predictor=predictor,
                num_samples=100
            )
            forecasts = list(forecast_deepar)
            tss = list(ts_deepar)

            forecasts_bin = {}
            for i in range(0, len(forecasts)):

                forecasts_bin[i] = np.where(forecasts[i].quantile(0.5) >= 0.5, 1, 0)
                daily_error += abs(np.where(forecasts_bin[i].sum() == 0, 0, 1) - np.where(tss[i][-24:].sum()[0] == 0, 0, 1))
                
        trial.set_user_attr(key='best_model', value=predictor)

        return daily_error
